In [1]:
import tensorflow as tf
import numpy as np

import sys
sys.path.append('./../../utils')
import model_conversion_tools as mct
from sample_pytorch_mnist import Net, train, test

### Part 1: sample Tensorflow project

sample project from our yolo test model

In [2]:
# Step 1: Load and Preprocess Fashion MNIST Data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
# Preprocess x_test
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255


#### training a sample tensorflow model:

To get the model for this example, feel free to look at https://github.com/UBCAgroBot/AppliedAI/blob/13-conversion-tools/23-I-12_SysArch/Test%20Models/YOLO_testmodel.py 

#### loading the model, converting to onnx and saving the model

In [3]:
# Later, if you want to load and evaluate the model without retraining
tf_model = tf.keras.models.load_model("fashion_mnist_model.h5")

tf_onnx_dest_path = ".\\tf_model.onnx"

onnx_sess = mct.tf_to_onnx(tf_model, tf_onnx_dest_path)

c:\Users\Valery\miniconda3\envs\cpsc330\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


#### Running the model

In [4]:
tf_predictions = tf_model.predict(x_test)
results_tf_ort = mct.predictOnnx(x_test, session=onnx_sess)
mct.checkPredictionConsistency(tf_predictions, results_tf_ort)
mct.checkConfidenceConsistency(tf_predictions, results_tf_ort)

313/313 [==============================] - 1s 2ms/step
All predictions are consistent
All confidence percentages are consistent


### Part 2: sample pyTorch workflow

#### creating sample pytorch model 

The example model is from this website: https://nextjournal.com/gkoehler/pytorch-mnist

In [ ]:
import torch
import torchvision
import torch.optim as optim

batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)


In [ ]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                                momentum=momentum)

for epoch in range(1,2):
  train(epoch)
  test()

network.eval()
torch_out_load = network(example_data)


Train Epoch: 1 [0/60000 (0%)]	Loss: 0.201695
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.402293
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.514808
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.204468
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.443682
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.483486
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.182736
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.376792
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.436870
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.322312
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.328916
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.346527
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.353329
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.643490
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.310386
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.170429
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.542867
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.262466
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.291557
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.175815
Train Epoch: 1 [12800/60000 (

c:\Users\Valery\miniconda3\envs\cpsc330\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.0907, Accuracy: 9714/10000 (97%)



In [ ]:
mct.torch_to_onnx(network, example_data, "torch_model.onnx")

ort_session = mct.loadOnnxModel("torch_model.onnx")
ort_predictions = mct.predictOnnx(example_data.numpy(), session=ort_session)

ort_predictions[0][0] = 3


c:\Users\Valery\Documents\GitHub\AppliedAI\23-I-12_SysArch\Experiments\valery_tests\conversion util demo\sample_pytorch_mnist.py:62: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
c:\Users\Valery\miniconda3\envs\cpsc330\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [ ]:
mct.checkPredictionConsistency(torch_out_load.detach().numpy(), ort_predictions)
mct.checkConfidenceConsistency(torch_out_load.detach().numpy(), ort_predictions)

Original: 1, ONNX: 0
[-2.4549718 -2.1185706 -2.4687483 -2.2076657 -2.3239005 -2.2315974
 -2.2770364 -2.3017619 -2.6148717 -2.1354787], 
[ 3.        -2.1185706 -2.4687483 -2.2076654 -2.3239005 -2.2315974
 -2.2770364 -2.3017619 -2.6148717 -2.1354787]
All predictions are consistent
Original: 	 [-2.4549718 -2.1185706 -2.4687483 -2.2076657 -2.3239005 -2.2315974
 -2.2770364 -2.3017619 -2.6148717 -2.1354787], 
ONNX: 	[ 3.        -2.1185706 -2.4687483 -2.2076654 -2.3239005 -2.2315974
 -2.2770364 -2.3017619 -2.6148717 -2.1354787]
All confidence percentages are consistent
